<div style="text-align:center; background-color: #f0f0f0; padding: 12px;">
    <h1 style="background-color: #007acc; color: white; padding: 10px;"><b>Red Neuronal: Self-Organizing Maps</b></h1>
    <p><strong>Materia:</strong> Análisis Exploratorio de Datos</p>
    <p><strong>Programa:</strong> Maestría en Ciencia de Datos e Información, INFOTEC</p>
    <p><strong>Docente:</strong> Dr. José Ortiz Béjar</p>
    <p><strong>Alumno:</strong> Rodrigo Guarneros Gutiérrez</p>
</div>

In [1]:
# Dependencias
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats as st
from sklearn.datasets  import make_blobs,make_circles
from sklearn import datasets
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import seaborn as sns; sns.set()

<div style="text-align:center; background-color: #f0f0f0; padding: 12px;">
    <h1 style="background-color: #007acc; color: white; padding: 10px;"><b>Resumen Ejecutivo</b></h1>

# Actividades

1. Para esta actividad se requiere el uso LabSOM (descargar del enlace).

2. Utiliza  un conjunto de datos de tu elección (puedes utilizar datos que ya hayas usado en actividades anteriores)

3. Utiliza una red con una arquitectura cuyas dimensiones (alto y ancho) tengan como cociente el mismo valor que se obtiene al dividir los dos primeros valores propios que se obtienen de un  análisis de componentes principales.

4. Entrena la red neuronal y según sea el caso utiliza métricas distintas a la euclidiana. Esto si a priori sabes que alguna otra métrica es más adecuada para tu conjunto de datos.

5. Aplica el método SOM Ward, exporta los agrupamientos obtenidos y evalúa cuál es el número óptimo de clusters.

6. Caracteriza los agrupamientos obtenidos utilizando los mapas de componentes.

7. Compara los resultados obtenidos con otros métodos de proyección y agrupamiento sobre los datos originales

8. Realiza un reporte de las estrategias que haya aplicado. El documento deberá incluir las siguientes secciones:


<h1 style="text-align:center; background-color: #007acc; color: white; padding: 10px;"><b>Introducción</b></h1>

Para el desarrollo de la red neuronal SOM se realizaron las siguientes actividades.

1. Para esta actividad se requiere el uso LabSOM, el cual se descargó del siguiente link: https://aulavirtual.infotec.mx/mod/url/view.php?id=58401.

Un software orgullosamente producido en el Laboratorio de Dinámica no Lineal de la Universidad Nacional Autónoma de México en 2014. 

Los datos que se pueden utilizar son datos en formato .txt. Si se cuenta con un file .csv, se debe guardar como un .txt. En este caso, se utilizan los datos de delitos en formato txt.

Su utilidad radica en la posibilidad de hacer análisis multivariado. Se puede decir que la metodología utilizada es sensible a la escala de las observaciones y la variación de las mismas, por lo que hay que normalizar las mismas. 

2. Utiliza  un conjunto de datos de tu elección (puedes utilizar datos que ya hayas usado en actividades anteriores)

Los datos que se utilizan corresponden al que corresponde e CAR FEATURES AND MSRP, disponible en la plataforma de Kaggle: https://www.kaggle.com/datasets/CooperUnion/cardataset?select=data.csv 

No obstante, estos datos deben pre-procesarse, en función de lo establecido en la metodología y tomando en cuenta que ésta depende de la escala de los datos y su variación.

3. Utiliza una red con una arquitectura cuyas dimensiones (alto y ancho) tengan como cociente el mismo valor que se obtiene al dividir los dos primeros valores propios que se obtienen de un  análisis de componentes principales.

<h1 style="text-align:center; background-color: #007acc; color: white; padding: 10px;"><b>Sección(es) que describen los análisis realizados y los resultados obtenidos para su dataset.
</b></h1>

### Pre-procesamiento

In [115]:
datos = pd.read_csv('autos.csv') 
datos2 = pd.read_csv('autos.csv') 


In [116]:
datos2['descripcion'] = datos2['Make']+datos2['Model']
datos2 = datos2[['descripcion'] + [col for col in datos2.columns if col != 'descripcion']]
datos2 = datos2.drop(columns=['Make', 'Model'])

datos2_agrupados = datos2.groupby(['descripcion']).mean()
datos2_agrupados

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP
descripcion,,,,,,,,
AcuraCL,2002.333333,236.666667,6.000000,2.000000,26.777778,17.000000,204.0,29818.888889
AcuraILX,2016.125000,191.437500,4.000000,4.000000,35.125000,24.625000,204.0,30712.500000
AcuraILX Hybrid,2014.000000,111.000000,4.000000,4.000000,38.000000,39.000000,204.0,31750.000000
AcuraIntegra,2000.208333,152.083333,4.000000,2.916667,28.166667,21.833333,204.0,11768.583333
AcuraLegend,1994.062500,215.000000,6.000000,3.250000,23.187500,16.000000,204.0,2143.500000
...,...,...,...,...,...,...,...,...
VolvoV90,1998.000000,181.000000,6.000000,4.000000,23.000000,16.000000,870.0,2200.000000
VolvoXC,2002.000000,197.000000,5.000000,4.000000,23.000000,17.000000,870.0,36500.000000
VolvoXC60,2015.481481,280.500000,4.851852,4.000000,27.222222,19.925926,870.0,43620.370370


In [118]:
datos2 = datos2.reset_index()
datos2 = datos2["descripcion"]
datos2

0        BMW1 Series M
1          BMW1 Series
2          BMW1 Series
3          BMW1 Series
4          BMW1 Series
             ...      
11909         AcuraZDX
11910         AcuraZDX
11911         AcuraZDX
11912         AcuraZDX
11913    LincolnZephyr
Name: descripcion, Length: 11914, dtype: object

In [121]:
datos2.to_csv('etiquetas.csv', sep=";", index=False)

In [92]:
# Fusionemos el modelo y la marca para después agrupar por lo que llamaremos "descripción"
datos['descripcion'] = datos['Make']+ ' ' + datos['Model']
datos = datos.drop(columns=['Make', 'Model'])
datos = datos[['descripcion'] + [col for col in datos.columns if col != 'descripcion']]
datos

,descripcion,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP
0,BMW 1 Series M,2011,335.0,6.0,2.0,26,19,3916,46135
1,BMW 1 Series,2011,300.0,6.0,2.0,28,19,3916,40650
2,BMW 1 Series,2011,300.0,6.0,2.0,28,20,3916,36350
3,BMW 1 Series,2011,230.0,6.0,2.0,28,18,3916,29450
4,BMW 1 Series,2011,230.0,6.0,2.0,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...
11909,Acura ZDX,2012,300.0,6.0,4.0,23,16,204,46120
11910,Acura ZDX,2012,300.0,6.0,4.0,23,16,204,56670
11911,Acura ZDX,2012,300.0,6.0,4.0,23,16,204,50620
11912,Acura ZDX,2013,300.0,6.0,4.0,23,16,204,50920


In [93]:
datos_agrupados2 = datos2.groupby("descripcion").mean()

In [81]:
datos_agrupados = datos.groupby("descripcion").mean()

In [82]:
datos_agrupados['Year'] = pd.to_datetime(datos_agrupados['Year'].round().astype(int), format='%Y')
datos_agrupados

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP
descripcion,,,,,,,,
Acura CL,2002-01-01,236.666667,6.000000,2.000000,26.777778,17.000000,204.0,29818.888889
Acura ILX,2016-01-01,191.437500,4.000000,4.000000,35.125000,24.625000,204.0,30712.500000
Acura ILX Hybrid,2014-01-01,111.000000,4.000000,4.000000,38.000000,39.000000,204.0,31750.000000
Acura Integra,2000-01-01,152.083333,4.000000,2.916667,28.166667,21.833333,204.0,11768.583333
Acura Legend,1994-01-01,215.000000,6.000000,3.250000,23.187500,16.000000,204.0,2143.500000
...,...,...,...,...,...,...,...,...
Volvo V90,1998-01-01,181.000000,6.000000,4.000000,23.000000,16.000000,870.0,2200.000000
Volvo XC,2002-01-01,197.000000,5.000000,4.000000,23.000000,17.000000,870.0,36500.000000
Volvo XC60,2015-01-01,280.500000,4.851852,4.000000,27.222222,19.925926,870.0,43620.370370


In [83]:
# Assuming 'datos_agrupados' is your DataFrame
datos_agrupados['Year_ok'] = datos_agrupados['Year'].dt.year
datos_agrupados.drop(columns=['Year'], inplace=True)

In [84]:
datos_agrupados = datos_agrupados.reset_index()
datos_agrupados

,descripcion,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP,Year_ok
0,Acura CL,236.666667,6.000000,2.000000,26.777778,17.000000,204.0,29818.888889,2002
1,Acura ILX,191.437500,4.000000,4.000000,35.125000,24.625000,204.0,30712.500000,2016
2,Acura ILX Hybrid,111.000000,4.000000,4.000000,38.000000,39.000000,204.0,31750.000000,2014
3,Acura Integra,152.083333,4.000000,2.916667,28.166667,21.833333,204.0,11768.583333,2000
4,Acura Legend,215.000000,6.000000,3.250000,23.187500,16.000000,204.0,2143.500000,1994
...,...,...,...,...,...,...,...,...,...
923,Volvo V90,181.000000,6.000000,4.000000,23.000000,16.000000,870.0,2200.000000,1998
924,Volvo XC,197.000000,5.000000,4.000000,23.000000,17.000000,870.0,36500.000000,2002
925,Volvo XC60,280.500000,4.851852,4.000000,27.222222,19.925926,870.0,43620.370370,2015
926,Volvo XC70,252.777778,4.944444,4.000000,27.500000,20.166667,870.0,40877.777778,2015


### Normalización

In [85]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

columnas = ['Year_ok', 'Engine HP', 'Engine Cylinders', 'Number of Doors', 'highway MPG', 'city mpg', 'Popularity', 'MSRP']

datos_agrupados[columnas] = scaler.fit_transform(datos_agrupados[columnas])

datos_agrupados


,descripcion,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP,Year_ok
0,Acura CL,0.185146,0.375000,0.000000,0.150794,0.078125,0.035721,0.015849,0.444444
1,Acura ILX,0.136927,0.250000,1.000000,0.235969,0.137695,0.035721,0.016358,0.962963
2,Acura ILX Hybrid,0.051173,0.250000,1.000000,0.265306,0.250000,0.035721,0.016949,0.888889
3,Acura Integra,0.094972,0.250000,0.458333,0.164966,0.115885,0.035721,0.005565,0.370370
4,Acura Legend,0.162047,0.375000,0.625000,0.114158,0.070312,0.035721,0.000082,0.148148
...,...,...,...,...,...,...,...,...,...
923,Volvo V90,0.125800,0.375000,1.000000,0.112245,0.070312,0.153492,0.000114,0.296296
924,Volvo XC,0.142857,0.312500,1.000000,0.112245,0.078125,0.153492,0.019656,0.444444
925,Volvo XC60,0.231876,0.303241,1.000000,0.155329,0.100984,0.153492,0.023712,0.925926
926,Volvo XC70,0.202322,0.309028,1.000000,0.158163,0.102865,0.153492,0.022150,0.925926


In [86]:
datos_agrupados.to_csv('automoviles2.csv', sep=';', index=False)

<h1 style="text-align:center; background-color: #007acc; color: white; padding: 10px;"><b>Comparación de Ward SOM con otros métodos de proyección y clustering
</b></h1>


Se utiliza el software LabSOM para una red neuronal de 3:1 y 1000 iteraciones para las siguientes características: 

1. Caballos de fuerza (Engine HP)
2. Número de cilindros (Engine Cylinders)
3. Número de puertas (Number of Doors)
4. Millas por galón en carretera (Highway MPG)
5. Millas por galón en ciudad (city MPG)
6. Popularidad del automóvil (Popularity)
7. Precio al menudeo recomendado por el manufacturero (MSRP)

Como resultado de la red neuronal y la aplicación de la técnica SOM se tienen los siguientes Mapas de calor de las 300 neuronas utilzadas en este modelo:

### Clusters

![clusters](clustering.png)

### Componentes
#### Popularidad y Precio recomendado
![clusters](popularity.png)
![clusters](precio_recomendado.png)

#### Caballos de fuerza y Cilindraje
![clusters](Engine_hp.png)
![clusters](Engine_cylinders.png)

#### Millas por galón de combustible en ciudad y carretera
![clusters](city_mpg.png)
![clusters](highway_mpg.png)

#### Número de puertas y año modelo
![clusters](number_doors.png)
![clusters](year.png)




<h1 style="text-align:center; background-color: #007acc; color: white; padding: 10px;"><b>Conclusiones
</b></h1>

1. La red neuronal y su visualización a través de LabSOM nos permite concluir que los precios de los automóviles están correlacionados esencialmente con el rendimiento del motor medido en millas por galón de combustible en ciudad y carretera.

2. Los automóviles con mayor precio tienen más cilindraje y más número de puertas. El año-modelo no parece ser un driver relevante.  

3. El rendimiento en ciudad y en carretera son dos variables con correlaciones positivas muy claras. Se podría decir que no es necesario tener ambas características en un modelo predictivo, bastaría con alguna de ellas.

4. El número de puertas también es un indicador del tamaño del automóvil y también su precio.

<h1 style="text-align:center; background-color: #007acc; color: white; padding: 10px;"><b>Referencias
</b></h1>

[1] Pellegrin, L., Loyola-González, O., Ortiz-Bejar, J., Medina-Pérez, M. A., Gutiérrez-Rodríguez, A. E., Tellez, E. S., ... & Esclalante, H. J. (2019). Image annotation as Text-Image matching: Challenge design and results. Computación y Sistemas, 23(4). doi: 10.13053/CyS-23-4-3207
[2] Van der Maaten, L., & Hinton, G. (2008). Visualizing data using t-SNE. Journal of machine learning research, 9(11).
[3] Talwalkar, A., Kumar, S., Morhri, M., & Rowley, H. A. (2013). Large Scale SVD and Manifold Learning.
[4] Humberto Carrillo Calvet, José Luis Jiménez Andrade, Elio A. Villaseñor García (2014). Software Lab SOM.  Departamento de Matemáticas, Laboratorio de Dinámica no Lineal de la Facultad de Ciencias de la UNAM.


